# The full python workflow for semasiological token-level clouds 

This notebook walks you through all the steps, from setting up the code based on your corpus to getting token-level distance matrices. They can be based on individual lemmas (like [here](https://cloudspotting.marianamontes.me/)) or on groups of related lemmas (like [here](https://www.degruyter.com/document/doi/10.1515/9783110733945-021/html)).

## 0. Initial setup 

The first step is to load some packages, including both `nephosem` and `semasioFlow`. Then we export functions directly from `semasioFlow` (although `ConfigLoader` is extracted from `nephosem`).

In [1]:
import os
import sys
import logging
sys.path.append("../../nephosem/") # my path to nephosem
sys.path.append("../semasioFlow/") # my path to semasioFlow

In [2]:
from semasioFlow import ConfigLoader
from semasioFlow.load import loadVocab, loadMacro, loadColloc, loadFocRegisters
from semasioFlow.sample import sampleTypes
from semasioFlow.focmodels import createBow, createRel, createPath
from semasioFlow.socmodels import targetPPMI, weightTokens, createSoc
from semasioFlow.utils import plotPatterns

## 1. Configuration 

Depending on what you need, you will have to set up some useful paths settings.
I like to have at least the path to my project (`mydir`), an output path within (`mydir + "output"`) and a GitHub path for the datasets that I will use in the visualization. There is no real reason not to have everything together, except that I did not think of it at the moment. (Actually, there is: the GitHub stuff will be public and huge data would not be included. How much do we want to have public?)

In [3]:
mydir = "./"
output_path = f"{mydir}/output/create-clouds/"
nephovis_path = f"{mydir}/for-nephovis/"
logging.basicConfig(filename = f'{mydir}/testlog.log', level = logging.DEBUG)

In [4]:
necessary_subfolders = ['vocab', 'cws', 'registers', 'tokens']
for sf in necessary_subfolders:
    if not os.path.exists(output_path + sf):
        os.makedirs(output_path + sf)

The variables with paths is just meant to make it easier to manipulate filenames. The most important concrete step is to adapt the configuration file.

In [5]:
conf = ConfigLoader()
settings = conf.update_config('config.ini')
settings['output-path'] = output_path

corpus_name = 'Toy'
print(settings['line-machine'])
print(settings['global-columns'])
print(settings['type'], settings['colloc'], settings['token'])

([^\t]+)\t([^\t])[^\t]*\t([^\t]+)\t([^\t]+)\t([^\t]+)\t([^\t]+)
word,pos,lemma,id,head,deprel
lemma/pos lemma/pos lemma/pos/fid/lid


## 2. Frequency lists

The frequency lists are the first thing to create, but once you have them, you just load them. So what we are going to do here is define the filename where we *would* store the frequency list (in this case, where it is actually stored), and if it exists it loads it; if it doesn't, it creates and store it. After generating a full frequency list, we might want to [filter it](https://qlvl.github.io/nephosem/tutorials/vocab.html) and store different versions in the `vocab` folder.

In [6]:
full_name = f"{output_path}/vocab/{corpus_name}.nodefreq"
print(full_name)
full = loadVocab(full_name, settings)
full

.//output/create-clouds//vocab/Toy.nodefreq


[('the/D', 53),('boy/N', 25),('eat/V', 22) ... ('ten/C', 1),('ask/V', 1),('about/I', 1)]

## 3. Boolean token-level matrices

Even though we first think of the type leven and only afterwards of the token level, with this workflow we don't really need to touch type level until after we obtain the boolean token-level matrices, that is, until we need to use PPMI values to select or weight the context words.

As a first step, we need the type or list of types we want to run; for example `"heet/adj"` or `["vernietig/verb", "verniel/verb"]`, and we subset the vocabulary for that query.

In [7]:
query = full.subvocab(["girl/N", "boy/N"])
type_name = "child"
query

[('girl/N', 21),('boy/N', 25)]

We could generate the tokens for all tokens (which in a real sample could be in the thousands), or create a random selection with a certain number and then only use those files. The output of `semasioFlow.sampleTypes()` includes a list of token IDs as well as the list of filenames that suffices to extract those tokens. We can then use the new list of filenames when we collect tokens, and the list of tokens to subset the resulting matrices. In addition, the optional argument `concordance` can take a filename to which a raw concordance based on the current settings can be stored (see [this tutorial](https://qlvl.github.io/nephosem/tutorials/concordance-from-tokens.html)).

Of course, to keep the sample fixed it would be more useful to generate the list, store it and then retrieve it in future runs.

<div class="alert alert-info">

<strong>NOTE:</strong> 
By default, `semasioFlow.sampleTypes()` will only sample one token of each type per file. To override this, add `oneperfile = False`.

</div>

In [8]:
import json
import os.path

fnames = [settings['corpus-path'] + '/' + x for x in os.listdir(settings['corpus-path'])]
tokenlist_fname = f"{mydir}/filelist.json"
if os.path.exists(tokenlist_fname):
    with open(tokenlist_fname, "r") as f:
        tokenlist, fnameSample = json.load(f).values()
else:
    tokenlist, fnameSample = sampleTypes({'girl/N' : 20, 'boy/N' : 20}, fnames, settings, oneperfile = False)
    with open(tokenlist_fname, "w") as f:
        json.dump({"tokenlist" : tokenlist, "fnames" : fnameSample}, f)

In [9]:
len(tokenlist)

40

In [10]:
len(fnameSample)

11

### 3.1 Bag-of-words

The code to generate one matrix is very straightforward, but what if we want to use different combinations of parameter settings to create multiple matrices?

The code below assumes that the boolean BOW matrices may vary across three parameters:

- **foc_win**: window size, which is set with numbers for let and right window. *This has the settings above for default*
- **foc_pos**: part-of-speech filter, which will actually be set as a previously filtered list of context words. *By default, all context words are included.*
- **bound**: the match for sentence boundaries and whether the models respond to them or not. *By default, sentence boundaries are ignored.*

In [11]:
nounverbs = [x for x in full.get_item_list() if x.rsplit("/", 1)[1] in ["N", "V"]]

In [12]:
foc_win = [(3, 3), (5, 5), (5, 3)] # optional window sizes
foc_pos = {
    "all" : full[full.freq > 2].get_item_list(), # only frequency filter
    "nounverbs" : nounverbs # only nouns and verbs
}
bound = { "match" : "^</s>$", "values" : [True, False]}

The function below combines a number of necessary functions:

- it creates a loop over the different combinations of parameter settings specified
- it collects the tokens and computes and filters the corresponding matrices
- it transforms the matrices in "boolean" integer matrices, with only 0's and 1's
- it stores the matrices in their respective files
- it records the combinations of parameter settings and which values are taken by each model
- it records the context words captured by each model for each token
- it returns both records to be stored wherever you want

In [13]:
bowdata = createBow(query, settings, type_name = type_name, fnames = fnameSample, tokenlist = tokenlist,
        foc_win = foc_win, foc_pos = foc_pos, bound = bound)
bowdata.to_csv(f"{output_path}/registers/{type_name}.bow-models.tsv", sep = "\t", index_label = "_model")

  0%|          | 0/6 [00:00<?, ?it/s]

Scanning tokens of queries in corpus...

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child//child.bound3-3all.tcmx.bool.pac

Saving matrix...


 17%|█▋        | 1/6 [00:00<00:01,  3.35it/s]

Stored in file:
  .//output/create-clouds//tokens/child//child.bound3-3nounverbs.tcmx.bool.pac
Scanning tokens of queries in corpus...

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child//child.nobound3-3all.tcmx.bool.pac

Saving matrix...


 33%|███▎      | 2/6 [00:00<00:01,  3.44it/s]

...

Stored in file:
  .//output/create-clouds//tokens/child//child.nobound5-5nounverbs.tcmx.bool.pac
Scanning tokens of queries in corpus...

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child//child.bound5-3all.tcmx.bool.pac

Saving matrix...


 83%|████████▎ | 5/6 [00:01<00:00,  3.45it/s]

Stored in file:
  .//output/create-clouds//tokens/child//child.bound5-3nounverbs.tcmx.bool.pac
Scanning tokens of queries in corpus...

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child//child.nobound5-3all.tcmx.bool.pac

Saving matrix...


100%|██████████| 6/6 [00:01<00:00,  3.49it/s]

Stored in file:
  .//output/create-clouds//tokens/child//child.nobound5-3nounverbs.tcmx.bool.pac


In [14]:
bowdata

,bound,foc_base,foc_pos,foc_win
child.bound3-3all,True,BOW,all,3-3
child.bound3-3nounverbs,True,BOW,nounverbs,3-3
child.bound5-3all,True,BOW,all,5-3
child.bound5-3nounverbs,True,BOW,nounverbs,5-3
child.bound5-5all,True,BOW,all,5-5
child.bound5-5nounverbs,True,BOW,nounverbs,5-5
child.nobound3-3all,False,BOW,all,3-3
child.nobound3-3nounverbs,False,BOW,nounverbs,3-3
child.nobound5-3all,False,BOW,all,5-3
child.nobound5-3nounverbs,False,BOW,nounverbs,5-3


### 3.2 Lemmarel

For dependency models we need specific templates and and patterns --- especially for LEMMAREL, they need to be tailored to the part-of-speech that you are looking into. Since I'm exemplifying with a verb, I will use those templates.

<div class="alert alert-danger">

<strong>IMPORTANT</strong><br>
In order to identify different sentences, dependency models require the 'separator-line-machine' value.
</div>

In [15]:
graphml_name = "LEMMAREL"
templates_dir = f"{mydir}/templates"
rel_macros = [
    ("LEMMAREL", loadMacro(templates_dir, graphml_name, "LEMMAREL")) # list of templates -we only have one
]
settings['separator-line-machine'] = "^</s>$"

In [16]:
reldata = createRel(query, settings, rel_macros, type_name = type_name,
              fnames = fnameSample, tokenlist = tokenlist, foc_filter = full.get_item_list())
reldata.to_csv(f"{output_path}/registers/{type_name}.rel-models.tsv", sep = "\t", index_label = "_model")
reldata

Building dependency features...



Building matrix...

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child//child.LEMMAREL.tcmx.bool.pac


,LEMMAREL,foc_base
child.LEMMAREL,LEMMAREL,LEMMAREL


### 3.3 Lemmapath

Like LEMMAREL, the LEMMAPATH models need 'separator-line-machine' to be properly set and the templates to be loaded.
Unlike LEMMAREL, the templates are not cumulative: LEMMAPATH1 models only cover those with one step between target and context word, while LEMMAPATH2 covers those with two steps. We *could* make them cumulative, but this setup allows us to give them different weights in PATHweight models.

In [17]:
graphml_name = "LEMMAPATH"
templates_dir = f"{mydir}/templates"
# If there were multiple LEMMAPATH templates with increasing path length:
# path_templates = [loadMacro(templates_dir, graphml_name, f"LEMMAPATH{i}") for i in [1, 2, 3]]
# path_macros = [
#     # First group includes templates with one and two steps, no weight
#     ("LEMMAPATH2", [path_templates[0], path_templates[1]], None),
#     # Second group includes templates with up to three steps, no weight
#     ("LEMMAPATH3", [path_templates[0], path_templates[1], path_templates[2]], None),
#     # Third group includes templates with up to three steps, with weight
#     ("LEMMAPATHweight", [path_templates[0], path_templates[1], path_templates[2]], [1, 0.6, 0.3])
# ]
path_templates = [loadMacro(templates_dir, graphml_name, "LEMMAPATH")]
path_macros = [("LEMMAPATH", path_templates, None)]
settings['separator-line-machine'] = "^</s>$"

In [18]:
pathdata = createPath(query, settings, path_macros, type_name = type_name,
          fnames = fnameSample, tokenlist = tokenlist, foc_filter = full.get_item_list())
pathdata.to_csv(f"{output_path}/registers/{type_name}.path-models.tsv", sep = "\t", index_label = "_model")
pathdata

Building dependency features...



Building matrix...

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child//child.LEMMAPATH.tcmx.bool.pac


,LEMMAPATH,foc_base
child.LEMMAPATH,LEMMAPATH,LEMMAPATH


## 4 Weight or booleanize

Once we have our boolean token-by-feature matrices, we can start combining them with type-level matrices: first to weight them and then to obtain second-order features. These functions will require us to specify the directory where we store our token matrices (in case we want different directories).

### 4.1. Create/load collocation matrix
First of all, we need to have a collocation matrix. The following function checks if the given filename exists and, if it doesn't, it creates the matrix from scratch.

In [19]:
freq_fname_CW4 = f"{output_path}/{corpus_name}.bow.wcmx.pac" # window size of 4

In [20]:
#settings['left-span'] = 4
#settings['right-span = 4']
freqMTX_CW4 = loadColloc(freq_fname_CW4, settings, row_vocab = full)
freqMTX_CW4

[55, 55]  's/P  ,/,  a/D  about/I  about/R  all/P  an/D  ...
's/P      NaN   NaN  NaN  NaN      NaN      NaN    NaN   ...
,/,       NaN   1    2    NaN      NaN      NaN    NaN   ...
a/D       NaN   3    NaN  NaN      NaN      NaN    NaN   ...
about/I   NaN   NaN  NaN  NaN      NaN      NaN    NaN   ...
about/R   NaN   NaN  NaN  NaN      NaN      NaN    NaN   ...
all/P     NaN   NaN  NaN  NaN      NaN      NaN    NaN   ...
an/D      NaN   NaN  NaN  NaN      NaN      NaN    NaN   ...
...       ...   ...  ...  ...      ...      ...    ...   ...

### 4.2 Register PPMI values

The function below subsets collocation matrices and calculates PMI values based on collocation matrices and frequencies based on vocabularies, to register the information in a dataframe. It returns a specific PPMI dataframe to use for weighting.

In [21]:
ppmi = targetPPMI(query.get_item_list(),
           vocabs = {"freq" : full},
          collocs = {"4" : freqMTX_CW4
#                      , "10" : freqMTX_CW10 # it's possible to add info from other matrices
                    },
          type_name = type_name, output_dir = f"{nephovis_path}/{type_name}/",
          main_matrix = "4" # matrix to base return matrix on
                 )
ppmi # it returns the PPMI values based on collocs["4"]



************************************
function    = compute_association
  time      = 0.01507 sec
************************************



[2, 48]  's/P       ,/,         a/D         about/I     about/R     all/P      an/D       ...
boy/N    NaN        NaN         NaN         NaN         NaN         0.8252069  0.6428853  ...
girl/N   1.0567893  0.09170843  0.46900266  0.83364576  0.83364576  NaN        NaN        ...

In [22]:
import pandas as pd
with open(f"{nephovis_path}/{type_name}/{type_name}.ppmi.tsv", "r") as f:
    pmidf = pd.read_csv(f, sep = "\t", index_col = "cw")
pmidf

,pmi_4_boy/N,pmi_4_girl/N,raw_4_boy/N,raw_4_girl/N,freq
cw,,,,,
's/P,NaN,1.056789,NaN,1.0,1
",/,",NaN,0.091708,NaN,2.0,3
a/D,-0.455727,0.469003,2.0,5.0,6
about/I,NaN,0.833646,NaN,1.0,1
about/R,NaN,0.833646,NaN,1.0,1
all/P,0.825207,NaN,1.0,NaN,1
an/D,0.642885,NaN,2.0,NaN,3
and/C,0.237420,-0.159606,3.0,2.0,5
apple/N,-0.621712,-0.207808,4.0,6.0,21


### 4.3 Implement weighting on selection

This step is performed on all the matrices created up to this moment. A useful thing to do first is to combine all the first-order register information we have from the different kinds of models.

In [23]:
registers = loadFocRegisters(f"{output_path}/registers/", type_name)
registers

,bound,foc_base,foc_pos,foc_win,LEMMAREL,LEMMAPATH
_model,,,,,,
child.bound3-3all,True,BOW,all,3-3,NaN,NaN
child.bound3-3nounverbs,True,BOW,nounverbs,3-3,NaN,NaN
child.bound5-3all,True,BOW,all,5-3,NaN,NaN
child.bound5-3nounverbs,True,BOW,nounverbs,5-3,NaN,NaN
child.bound5-5all,True,BOW,all,5-5,NaN,NaN
child.bound5-5nounverbs,True,BOW,nounverbs,5-5,NaN,NaN
child.nobound3-3all,False,BOW,all,3-3,NaN,NaN
child.nobound3-3nounverbs,False,BOW,nounverbs,3-3,NaN,NaN
child.nobound5-3all,False,BOW,all,5-3,NaN,NaN


Once we have the registers, we can also set the values for our `PPMI` setting with the `weighting` dictionary. A value `None` indicates that no weighting is applied, while matrices as values (a boolean version for selection instead of weighting, for example) are used to weight the tokens.

In [24]:
from semasioFlow.utils import booleanize
weighting = {
    "no" : None, # no weighting
    "selection" : booleanize(ppmi, include_negative=False), # select with ppmi
    "weight" : ppmi # weight with ppmi
}

In [25]:
token_dir = f"{output_path}/tokens/{type_name}"
weight_data = weightTokens(token_dir, weighting, registers)


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.bound3-3all.PPMIno.tcmx.weight.pac

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.bound3-3all.PPMIselection.tcmx.weight.pac

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.bound3-3all.PPMIweight.tcmx.weight.pac

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.bound3-3nounverbs.PPMIno.tcmx.weight.pac

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.bound3-3nounverbs.PPMIselection.tcmx.weight.pac

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.bound3-3nounverbs.PPMIweight.tcmx.weight.pac

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.bound5-3all.PPMIno.tcmx.weight.pac

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.bound5-3all.PPMIselection.tcmx.weight.pac

Saving matrix...
Stored in file:

In [26]:
# new model register
weight_data["model_register"]
# weight_data["model_register"].to_csv(f"{output_path}/registers/{type_name}.focmodels.tsv", sep = '\t',
# index_label = "_model")

,LEMMAPATH,LEMMAREL,bound,foc_base,foc_context_words,foc_pmi,foc_pos,foc_win,tokens
child.LEMMAPATH.PPMIno,LEMMAPATH,NaN,NaN,LEMMAPATH,23,no,NaN,NaN,40
child.LEMMAPATH.PPMIselection,LEMMAPATH,NaN,NaN,LEMMAPATH,20,selection,NaN,NaN,40
child.LEMMAPATH.PPMIweight,LEMMAPATH,NaN,NaN,LEMMAPATH,20,weight,NaN,NaN,40
child.LEMMAREL.PPMIno,NaN,LEMMAREL,NaN,LEMMAREL,10,no,NaN,NaN,32
child.LEMMAREL.PPMIselection,NaN,LEMMAREL,NaN,LEMMAREL,10,selection,NaN,NaN,26
child.LEMMAREL.PPMIweight,NaN,LEMMAREL,NaN,LEMMAREL,10,weight,NaN,NaN,26
child.bound3-3all.PPMIno,NaN,NaN,True,BOW,19,no,all,3-3,40
child.bound3-3all.PPMIselection,NaN,NaN,True,BOW,14,selection,all,3-3,40
child.bound3-3all.PPMIweight,NaN,NaN,True,BOW,14,weight,all,3-3,40
child.bound3-3nounverbs.PPMIno,NaN,NaN,True,BOW,16,no,nounverbs,3-3,40


In [27]:
# token_level register
weight_data["token_register"]

,_count.child.LEMMAPATH.PPMIno,_count.child.LEMMAPATH.PPMIselection,_count.child.LEMMAPATH.PPMIweight,_count.child.LEMMAREL.PPMIno,_count.child.LEMMAREL.PPMIselection,_count.child.LEMMAREL.PPMIweight,_count.child.bound3-3all.PPMIno,_count.child.bound3-3all.PPMIselection,_count.child.bound3-3all.PPMIweight,_count.child.bound3-3nounverbs.PPMIno,...,_cws.child.nobound5-3all.PPMIweight,_cws.child.nobound5-3nounverbs.PPMIno,_cws.child.nobound5-3nounverbs.PPMIselection,_cws.child.nobound5-3nounverbs.PPMIweight,_cws.child.nobound5-5all.PPMIno,_cws.child.nobound5-5all.PPMIselection,_cws.child.nobound5-5all.PPMIweight,_cws.child.nobound5-5nounverbs.PPMIno,_cws.child.nobound5-5nounverbs.PPMIselection,_cws.child.nobound5-5nounverbs.PPMIweight
boy/N/StanfDepSents.1/9,2,2,2,1.0,1.0,1.0,3,3,3,1,...,look/V;at/I;the/D,girl/N;look/V,look/V,look/V,at/I;girl/N;healthy/J;look/V;the/D,look/V;at/I;the/D,look/V;at/I;the/D,girl/N;look/V,look/V,look/V
boy/N/StanfDepSents.12/12,2,2,2,1.0,1.0,1.0,2,1,1,2,...,the/D;be/V,be/V;eat/V;like/V;year/N,like/V;be/V,like/V;be/V,an/D;apple/N;be/V;eat/V;tasty/J;the/D;year/N,an/D;the/D;be/V,an/D;the/D;be/V,apple/N;be/V;eat/V;like/V;year/N,like/V;be/V,like/V;be/V
boy/N/StanfDepSents.2/12,2,1,1,1.0,NaN,NaN,3,1,1,1,...,by/I;the/D,boy/N;eat/V,NaN,NaN,apple/N;boy/N;by/I;eat/V;food/N;healthy/J;the/D,by/I;the/D,by/I;the/D,apple/N;boy/N;eat/V;food/N,NaN,NaN
boy/N/StanfDepSents.2/8,2,2,2,NaN,NaN,NaN,3,2,2,1,...,by/I;the/D;be/V,apple/N;be/V;boy/N;eat/V,be/V,be/V,apple/N;be/V;boy/N;by/I;eat/V;healthy/J;the/D,by/I;the/D;be/V,by/I;the/D;be/V,apple/N;be/V;boy/N;eat/V,be/V,be/V
boy/N/StanfDepSents.3/18,4,2,2,1.0,NaN,NaN,3,2,2,1,...,and/C;the/D;be/V,apple/N;be/V;eat/V;girl/N;have/V,have/V;be/V,have/V;be/V,a/D;and/C;apple/N;be/V;eat/V;girl/N;the/D,and/C;the/D;be/V,and/C;the/D;be/V,apple/N;be/V;eat/V;girl/N;have/V,have/V;be/V,have/V;be/V
boy/N/StanfDepSents.3/3,2,1,1,1.0,NaN,NaN,3,2,2,1,...,an/D;the/D,eat/V,NaN,NaN,an/D;apple/N;eat/V;the/D,an/D;the/D,an/D;the/D,apple/N;eat/V,NaN,NaN
boy/N/StanfDepSents.4/22,2,2,2,1.0,1.0,1.0,2,2,2,3,...,give/V;the/D,apple/N;baby/N;give/V;have/V,baby/N;give/V;have/V,baby/N;give/V;have/V,apple/N;give/V;the/D,give/V;the/D,give/V;the/D,apple/N;baby/N;give/V;have/V,baby/N;give/V;have/V,baby/N;give/V;have/V
boy/N/StanfDepSents.4/3,2,2,2,1.0,1.0,1.0,1,1,1,1,...,the/D,say/V,say/V,say/V,girl/N;the/D,the/D,the/D,girl/N;say/V,say/V,say/V
boy/N/StanfDepSents.5/11,2,1,1,1.0,NaN,NaN,4,2,2,2,...,look/V;and/C;the/D,eat/V;look/V;sit/V,look/V,look/V,and/C;eat/V;look/V;sit/V;tasty/J;the/D,look/V;and/C;the/D,look/V;and/C;the/D,eat/V;look/V;sit/V,look/V,look/V
boy/N/StanfDepSents.5/25,2,2,2,NaN,NaN,NaN,3,3,3,1,...,by/I;give/V;the/D;be/V,be/V;eat/V;give/V,give/V;be/V,give/V;be/V,be/V;by/I;eat/V;give/V;the/D,by/I;give/V;the/D;be/V,by/I;give/V;the/D;be/V,be/V;eat/V;give/V,give/V;be/V,give/V;be/V


In [28]:
weight_data["token_register"].to_csv(f"{nephovis_path}/{type_name}/{type_name}.variables.tsv", sep = '\t', index_label = "_id")

### 5 Second-order dimensions

The final step to obtain our token-level vectors is to multiply the token-foc matrices for type-level matrices to obtain second-order vectors. We will loop over the models in the index of `weight_data["model_register"]` and over second-order parameter settings to filter `freqMTX_CW4` and obtain different models.

In [29]:
soc_pos = {
    "all" : full[full.freq > 2], # all possible dimensions
    "nv" : full.subvocab(nounverbs) # only nouns and verbs
}
lengths = ["FOC", 10] # a number will take the most frequent; something else will take the FOC items

In [30]:
socdata = createSoc(
    token_dir,
    registers = weight_data['model_register'],
    soc_pos = soc_pos, lengths = lengths,
    socMTX = freqMTX_CW4,
    store_focdists = f"{output_path}/cws/{type_name}/") # create distance matrices for context words



************************************
function    = compute_association
  time      = 0.01494 sec
************************************


************************************
function    = compute_distance
  time      = 0.0008421 sec
************************************

  Operation: addition 'token-feature weight matrix' X 'socc matrix'...

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.LEMMAPATH.PPMIno.LENGTHFOC.SOCPOSall.tcmx.soc.pac




************************************
function    = compute_association
  time      = 0.01419 sec
************************************


************************************
function    = compute_distance
  time      = 0.0003443 sec
************************************

  Operation: addition 'token-feature weight matrix' X 'socc matrix'...

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.LEMMAPATH.PPMIno.LENGTH10.SOCPOSall.tcmx.soc.pac




************************************
function    = compute_association
  time      = 0.01451 sec
************************************


************************************
function    = compute_distance
  time      = 0.0004005 sec
************************************

  Operation: addition 'token-feature weight matrix' X 'socc matrix'...

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.LEMMAPATH.PPMIno.LENGTHFOC.SOCPOSnv.tcmx.soc.pac




************************************
function    = compute_association
  time      = 0.014 sec
************************************


************************************
function    = compute_distance
  time      = 0.0004139 sec
************************************

  Operation: addition 'token-feature weight matrix' X 'socc matrix'...

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.LEMMAPATH.PPMIno.LENGTH10.SOCPOSnv.tcmx.soc.pac




************************************
function    = compute_association
  time      = 0.01402 sec
************************************


************************************
function    = compute_distance
  time      = 0.0003409 sec
************************************

  Operation: addition 'token-feature weight matrix' X 'socc matrix'...

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.LEMMAPATH.PPMIselection.LENGTHFOC.SOCPOSall.tcmx.soc.pac


...



************************************
function    = compute_association
  time      = 0.01434 sec
************************************


************************************
function    = compute_distance
  time      = 0.0002866 sec
************************************

  Operation: addition 'token-feature weight matrix' X 'socc matrix'...

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.nobound5-5nounverbs.PPMIselection.LENGTH10.SOCPOSnv.tcmx.soc.pac




************************************
function    = compute_association
  time      = 0.01368 sec
************************************


************************************
function    = compute_distance
  time      = 0.0002737 sec
************************************

  Operation: addition 'token-feature weight matrix' X 'socc matrix'...

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.nobound5-5nounverbs.PPMIweight.LENGTHFOC.SOCPOSall.tcmx.soc.pac




************************************
function    = compute_association
  time      = 0.01433 sec
************************************


************************************
function    = compute_distance
  time      = 0.0003326 sec
************************************

  Operation: addition 'token-feature weight matrix' X 'socc matrix'...

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.nobound5-5nounverbs.PPMIweight.LENGTH10.SOCPOSall.tcmx.soc.pac




************************************
function    = compute_association
  time      = 0.01373 sec
************************************


************************************
function    = compute_distance
  time      = 0.0002837 sec
************************************

  Operation: addition 'token-feature weight matrix' X 'socc matrix'...

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.nobound5-5nounverbs.PPMIweight.LENGTHFOC.SOCPOSnv.tcmx.soc.pac




************************************
function    = compute_association
  time      = 0.01407 sec
************************************


************************************
function    = compute_distance
  time      = 0.0003114 sec
************************************

  Operation: addition 'token-feature weight matrix' X 'socc matrix'...

Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.nobound5-5nounverbs.PPMIweight.LENGTH10.SOCPOSnv.tcmx.soc.pac


In [31]:
socdata

,LEMMAPATH,LEMMAREL,bound,foc_base,foc_context_words,foc_pmi,foc_pos,foc_win,soc_length,soc_pos,tokens
child.LEMMAPATH.PPMIno.LENGTH10.SOCPOSall,LEMMAPATH,NaN,NaN,LEMMAPATH,23,no,NaN,NaN,10,all,40
child.LEMMAPATH.PPMIno.LENGTH10.SOCPOSnv,LEMMAPATH,NaN,NaN,LEMMAPATH,23,no,NaN,NaN,10,nv,40
child.LEMMAPATH.PPMIno.LENGTHFOC.SOCPOSall,LEMMAPATH,NaN,NaN,LEMMAPATH,23,no,NaN,NaN,FOC,all,40
child.LEMMAPATH.PPMIno.LENGTHFOC.SOCPOSnv,LEMMAPATH,NaN,NaN,LEMMAPATH,23,no,NaN,NaN,FOC,nv,40
child.LEMMAPATH.PPMIselection.LENGTH10.SOCPOSall,LEMMAPATH,NaN,NaN,LEMMAPATH,20,selection,NaN,NaN,10,all,40
child.LEMMAPATH.PPMIselection.LENGTH10.SOCPOSnv,LEMMAPATH,NaN,NaN,LEMMAPATH,20,selection,NaN,NaN,10,nv,40
child.LEMMAPATH.PPMIselection.LENGTHFOC.SOCPOSall,LEMMAPATH,NaN,NaN,LEMMAPATH,20,selection,NaN,NaN,FOC,all,40
child.LEMMAPATH.PPMIselection.LENGTHFOC.SOCPOSnv,LEMMAPATH,NaN,NaN,LEMMAPATH,20,selection,NaN,NaN,FOC,nv,40
child.LEMMAPATH.PPMIweight.LENGTH10.SOCPOSall,LEMMAPATH,NaN,NaN,LEMMAPATH,20,weight,NaN,NaN,10,all,40
child.LEMMAPATH.PPMIweight.LENGTH10.SOCPOSnv,LEMMAPATH,NaN,NaN,LEMMAPATH,20,weight,NaN,NaN,10,nv,40


In [32]:
socdata.to_csv(f"{nephovis_path}/{type_name}/{type_name}.models.tsv", sep = "\t", index_label="_model")

### 6 Cosine distances
Once we have all the token-level vectors, as well as our registers,
we can quickly compute and store their cosine distances.

In [33]:
from nephosem import TypeTokenMatrix
from nephosem.specutils.mxcalc import compute_distance

input_suffix = ".tcmx.soc.pac" #token by context matrix
output_suffix = ".ttmx.dist.pac" # token by token matrix
for modelname in socdata.index:
    input_name = f"{token_dir}/{modelname}{input_suffix}"
    output_name = f"{token_dir}/{modelname}{output_suffix}"
    compute_distance(TypeTokenMatrix.load(input_name)).save(output_name)


************************************
function    = compute_distance
  time      = 0.000612 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.LEMMAPATH.PPMIno.LENGTH10.SOCPOSall.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.0004447 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.LEMMAPATH.PPMIno.LENGTH10.SOCPOSnv.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.0003285 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.LEMMAPATH.PPMIno.LENGTHFOC.SOCPOSall.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.0003374 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/chi

Stored in file:
  .//output/create-clouds//tokens/child/child.bound3-3all.PPMIselection.LENGTHFOC.SOCPOSall.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.000289 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.bound3-3all.PPMIselection.LENGTHFOC.SOCPOSnv.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.0002892 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.bound3-3all.PPMIweight.LENGTH10.SOCPOSall.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.0002916 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.bound3-3all.PPMIweight.LENGTH10.SOCPOSnv.ttmx.dist.pac

************************************
function    = compute_dist

Stored in file:
  .//output/create-clouds//tokens/child/child.bound5-3nounverbs.PPMIno.LENGTH10.SOCPOSall.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.0002878 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.bound5-3nounverbs.PPMIno.LENGTH10.SOCPOSnv.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.0002971 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.bound5-3nounverbs.PPMIno.LENGTHFOC.SOCPOSall.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.0002995 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.bound5-3nounverbs.PPMIno.LENGTHFOC.SOCPOSnv.ttmx.dist.pac

************************************
function    = compute_d

Stored in file:
  .//output/create-clouds//tokens/child/child.bound5-5nounverbs.PPMIselection.LENGTHFOC.SOCPOSall.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.0002847 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.bound5-5nounverbs.PPMIselection.LENGTHFOC.SOCPOSnv.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.0002799 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.bound5-5nounverbs.PPMIweight.LENGTH10.SOCPOSall.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.000288 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.bound5-5nounverbs.PPMIweight.LENGTH10.SOCPOSnv.ttmx.dist.pac

************************************
fu

Stored in file:
  .//output/create-clouds//tokens/child/child.nobound5-3all.PPMIno.LENGTH10.SOCPOSall.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.0002995 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.nobound5-3all.PPMIno.LENGTH10.SOCPOSnv.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.0003011 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.nobound5-3all.PPMIno.LENGTHFOC.SOCPOSall.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.0002873 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.nobound5-3all.PPMIno.LENGTHFOC.SOCPOSnv.ttmx.dist.pac

************************************
function    = compute_distance
  time  

Stored in file:
  .//output/create-clouds//tokens/child/child.nobound5-5all.PPMIselection.LENGTHFOC.SOCPOSall.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.0002856 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.nobound5-5all.PPMIselection.LENGTHFOC.SOCPOSnv.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.0002885 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.nobound5-5all.PPMIweight.LENGTH10.SOCPOSall.ttmx.dist.pac

************************************
function    = compute_distance
  time      = 0.0002856 sec
************************************


Saving matrix...
Stored in file:
  .//output/create-clouds//tokens/child/child.nobound5-5all.PPMIweight.LENGTH10.SOCPOSnv.ttmx.dist.pac

************************************
function    = com

For the rest, we go to R!

The R code is in the processClouds notebook, which uses the [semcloud](https://montesmariana.github.io/semcloud) package.

## Bonus: context word detail

In [34]:
from semasioFlow.contextwords import listContextwords

In [35]:
cws = listContextwords(type_name, tokenlist, fnameSample, settings, left_win=15, right_win = 15)

100%|██████████| 11/11 [00:00<00:00, 417.83it/s]


In [36]:
cws

,cw,deprel,distance,head,id,lemma,path,pos,position,rep_path,same_sentence,side,steps,target_lemma,token_id,word
boy/N/StanfDepSents.1/9/L0,the/D,det,1,2,1,the,#T->det:the,D,L0,#T->det:Cw,True,L,1,child,boy/N/StanfDepSents.1/9,The
boy/N/StanfDepSents.1/9/L1,NaN,"<s id=""2"">",2,"<s id=""2"">","<s id=""2"">","<s id=""2"">",NaN,"<s id=""2"">",L1,NaN,False,L,NaN,child,boy/N/StanfDepSents.1/9,"<s id=""2"">"
boy/N/StanfDepSents.1/9/L2,NaN,</s>,3,</s>,</s>,</s>,NaN,</s>,L2,NaN,False,L,NaN,child,boy/N/StanfDepSents.1/9,</s>
boy/N/StanfDepSents.1/9/L3,healthy/J,acomp,4,3,4,healthy,NaN,J,L3,NaN,False,L,NaN,child,boy/N/StanfDepSents.1/9,healthy
boy/N/StanfDepSents.1/9/L4,look/V,ROOT,5,0,3,look,NaN,V,L4,NaN,False,L,NaN,child,boy/N/StanfDepSents.1/9,looks
boy/N/StanfDepSents.1/9/L5,girl/N,nsubj,6,3,2,girl,NaN,N,L5,NaN,False,L,NaN,child,boy/N/StanfDepSents.1/9,girl
boy/N/StanfDepSents.1/9/L6,the/D,det,7,2,1,the,NaN,D,L6,NaN,False,L,NaN,child,boy/N/StanfDepSents.1/9,The
boy/N/StanfDepSents.1/9/L7,NaN,"<s id=""1"">",8,"<s id=""1"">","<s id=""1"">","<s id=""1"">",NaN,"<s id=""1"">",L7,NaN,False,L,NaN,child,boy/N/StanfDepSents.1/9,"<s id=""1"">"
boy/N/StanfDepSents.1/9/R0,look/V,ROOT,1,0,3,look,look->nsubj:#T,V,R0,Cw->nsubj:#T,True,R,1,child,boy/N/StanfDepSents.1/9,looks
boy/N/StanfDepSents.1/9/R1,at/I,prep,2,3,4,at,"look->[nsubj:#T,prep:at]",I,R1,"look->[nsubj:#T,prep:Cw]",True,R,2,child,boy/N/StanfDepSents.1/9,at


In [37]:
cw_fname = f"{nephovis_path}/{type_name}/{type_name}.cws.detail.tsv"
cws.to_csv(cw_fname, sep = "\t", index_label = "cw_id")

From this table, it is relatively straightforward to extract concordances and highlight the context words that match certain filters. Note that by default the left contexts are in reverse order.